In [5]:
# CÉLULA 1: PREPARAÇÃO DO AMBIENTE E CARREGAMENTO DOS DADOS

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error

# Carregamento e cópia do dataset students_performance.csv
caminho_arquivo = '../data/datasets/students_performance.csv' 
df = pd.read_csv(caminho_arquivo)
df_prep = df.copy()

# PRÉ-PROCESSAMENTO CONSOLIDADO
# Imputação (Média e Moda)
colunas_categoricas_com_nan = ['extracurricular', 'internet_quality', 'family_income', 'health_status']
for coluna in colunas_categoricas_com_nan:
    moda = df_prep[coluna].mode()[0]
    df_prep[coluna].fillna(moda, inplace=True)
df_prep['study_hours_week'].fillna(df_prep['study_hours_week'].mean(), inplace=True)
df_prep['sleep_hours'].fillna(df_prep['sleep_hours'].mean(), inplace=True)
df_prep.dropna(inplace=True) # Linha de segurança para NaN remanescentes

# One-Hot Encoding e Remoção de IDs
colunas_categoricas = df_prep.select_dtypes(include='object').columns.tolist()
df_prep = pd.get_dummies(df_prep, columns=colunas_categoricas, drop_first=True)
colunas_id = [col for col in df_prep.columns if col.startswith('student_id')]
df_prep = df_prep.drop(columns=colunas_id, axis=1)

# SEPARAÇÃO X e y, ESCALONAMENTO e TRAIN/TEST SPLIT
target_column = 'final_grade'
X = df_prep.drop(columns=[target_column], axis=1)
y = df_prep[target_column]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42
)
print("Dados prontos. Dimensões de Treino:", X_train.shape)

Dados prontos. Dimensões de Treino: (1728, 56)


In [10]:
# CÉLULA 2: GRID SEARCH E OTIMIZAÇÃO DE HYPERPARÂMETROS

# 1. DEFINIR O MODELO BASE
rf_model = RandomForestRegressor(random_state=42)

# 2. DEFINIR O ESPAÇO DE PARÂMETROS PARA TESTAR
param_grid = {
    'n_estimators': [100, 200],      
    'max_depth': [None, 15, 25],      
    'min_samples_split': [2, 5]     
}

# 3. CONFIGURAR O GRID SEARCH
grid_search = GridSearchCV(
    estimator=rf_model, 
    param_grid=param_grid, 
    scoring='r2', 
    cv=3, 
    verbose=1, 
    n_jobs=-1 
)

# 4. EXECUTAR O GRID SEARCH (TREINAMENTO OTIMIZADO)
print("\nIniciando Grid Search para otimização...")
grid_search.fit(X_train, y_train)

# 5. EXIBIR OS MELHORES RESULTADOS
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_rf_model = grid_search.best_estimator_

print(f"\nMelhores Parâmetros Encontrados: {best_params}")
print(f"Melhor R² Médio na Validação Cruzada: {best_score:.4f}")

# 6. AVALIAÇÃO FINAL NO CONJUNTO DE TESTE
y_pred_final = best_rf_model.predict(X_test)
final_r2 = r2_score(y_test, y_pred_final)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))

print(f"\n--- AVALIAÇÃO FINAL NO CONJUNTO DE TESTE ---")
print(f"R² Final (Modelo Otimizado): {final_r2:.4f}")
print(f"RMSE Final (Modelo Otimizado): {final_rmse:.4f}")


Iniciando Grid Search para otimização...
Fitting 3 folds for each of 12 candidates, totalling 36 fits

Melhores Parâmetros Encontrados: {'max_depth': 25, 'min_samples_split': 2, 'n_estimators': 200}
Melhor R² Médio na Validação Cruzada: 0.6665

--- AVALIAÇÃO FINAL NO CONJUNTO DE TESTE ---
R² Final (Modelo Otimizado): 0.6345
RMSE Final (Modelo Otimizado): 4.4220


In [11]:
# CÉLULA 3: SALVAMENTO DO MODELO FINAL

import joblib
import os # Importação necessária para criar pastas

caminho_modelo = '../models/modelo_final.joblib'
diretorio_modelo = '../models/'

# 1. GARANTIR QUE O DIRETÓRIO EXISTA
os.makedirs(diretorio_modelo, exist_ok=True)
print(f"Diretório '{diretorio_modelo}' verificado/criado.")

# 2. Salvar o modelo vencedor do Grid Search (best_rf_model)
joblib.dump(best_rf_model, caminho_modelo)

print(f"Modelo otimizado salvo com sucesso em: {caminho_modelo}")

Diretório '../models/' verificado/criado.
Modelo otimizado salvo com sucesso em: ../models/modelo_final.joblib
